In [1]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd drive/MyDrive

/content/drive/MyDrive


In [5]:
myFile = open("distance_matrix.txt")
data = myFile.read()
data = data.replace('[', '')
data = data.replace(']', '')
dataList = data.split("\n")
distance_matrix_str = []
for i in dataList:
    i = i.replace(' ', '')
    listTmp = i.split(',')
    distance_matrix_str.append(listTmp)

distance_matrix = []
for i in distance_matrix_str:
    distance_matrix.append([int(j) for j in i])
print(len(distance_matrix))

myFile.close()

195


In [6]:
mat1 = np.array(distance_matrix)

In [7]:
print(mat1)

[[    0  1799  3527 ... 19076 17795 15058]
 [ 1799     0  2528 ... 17527 16246 13509]
 [ 3527  2528     0 ... 17138 15858 13121]
 ...
 [19076 17527 17138 ...     0  5693  4037]
 [17795 16246 15858 ...  5693     0  4793]
 [15058 13509 13121 ...  4037  4793     0]]


# First approach

In [8]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [41]:
class ORTools_Optimizer:
    #def __init__(self, mat, vehicles):
    def __init__(self, mat, vehicles, points):
        self.mat = mat
        self.vehicles = vehicles
        self.points = points
        self.data = {}
    
    def create_data_model(self):
        """Stores the data for the problem."""
        self.data['distance_matrix'] = self.mat
        self.data['num_vehicles'] = self.vehicles
        self.data['depot'] = 0
    
    def print_solution(self, data, manager, routing, solution):
        """Prints solution on console."""
        print(f'Objective: {solution.ObjectiveValue()}')
        max_route_distance = 0
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
            route_distance = 0
            while not routing.IsEnd(index):
                plan_output += ' {} -> '.format(self.points[manager.IndexToNode(index)])
                #plan_output += ' {} -> '.format(manager.IndexToNode(index))
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)
            plan_output += '{}\n'.format(manager.IndexToNode(index))
            plan_output += 'Distance of the route: {}m\n'.format(route_distance)
            print(plan_output)
            max_route_distance = max(route_distance, max_route_distance)
        print('Maximum of the route distances: {}m'.format(max_route_distance))
        
    
    def distance_callback(self, from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = self.manager.IndexToNode(from_index)
        to_node = self.manager.IndexToNode(to_index)
        return self.data['distance_matrix'][from_node][to_node]
    
    def run(self):
        self.create_data_model()

        self.manager = pywrapcp.RoutingIndexManager(len(self.data['distance_matrix']), self.data['num_vehicles'], self.data['depot'])

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(self.manager)
        transit_callback_index = routing.RegisterTransitCallback(self.distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Add Distance constraint.
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            30000000,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(10000000)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)

        # Print solution on console.
        if solution:
            self.print_solution(self.data, self.manager, routing, solution)
        else:
            print('No solution found !')

In [40]:
points = np.zeros(195)
for i in range(195):
    points[i] = i
optimizer = ORTools_Optimizer(mat1, 5, points)
optimizer.run()

Objective: 556860272104
Route for vehicle 0:
 0 ->  102 ->  113 ->  98 ->  103 ->  97 ->  137 ->  126 ->  95 ->  100 ->  96 ->  99 ->  129 ->  105 ->  123 ->  106 ->  101 ->  104 ->  107 ->  108 ->  110 ->  122 ->  118 ->  130 ->  135 ->  116 ->  131 ->  132 ->  133 ->  119 ->  125 ->  120 ->  139 ->  121 ->  109 ->  79 ->  54 ->  60 ->  62 ->  50 ->  61 ->  63 ->  70 ->  82 ->  88 ->  86 ->  67 ->  81 ->  83 ->  73 ->  93 ->  89 ->  85 ->  76 ->  59 ->  64 ->  68 ->  69 ->  51 ->  84 ->  72 ->  56 ->  55 ->  77 ->  66 ->  47 ->  12 ->  10 ->  4 ->  23 ->  41 ->  33 ->  38 ->  36 ->  35 ->  34 ->  32 ->  43 ->  8 ->  26 ->  39 ->  22 ->  27 ->  5 ->  18 ->  29 ->  28 ->  158 -> 0
Distance of the route: 53877m

Route for vehicle 1:
 0 ->  25 ->  87 ->  49 ->  58 ->  52 ->  74 ->  90 ->  92 ->  53 ->  182 ->  184 ->  190 ->  154 ->  178 ->  180 ->  167 ->  172 ->  173 ->  161 ->  166 ->  177 ->  163 ->  174 ->  185 ->  175 ->  192 ->  170 ->  183 ->  57 ->  75 ->  94 ->  78 ->  80 ->  91

# Second approach
https://arxiv.org/pdf/1812.02300.pdf

In [42]:
from sklearn.cluster import DBSCAN

def num_clusters(labels):
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)

    return n_clusters, n_noise

def cluster_size(labels):
    labelDict = {}
    for i in labels:
        if i in labelDict:
            labelDict[i] += 1
        else:
            labelDict[i] = 1
    
    max_cluster_size = 0
    avg_cluster_size = 0
    for k,v in labelDict.items():
        if k == -1:
            continue
        max_cluster_size = max(max_cluster_size, v)
        avg_cluster_size += v
    
    n_clusters, _ = num_clusters(labels)
    avg_cluster_size /= n_clusters
    return max_cluster_size, avg_cluster_size

m_per_radian = 6371008.8
best_avg_cluster_size = 0
min_radius = 1
max_radius = 5e15
max_cluster_size_const = 90
final_clusters = None
while min_radius < max_radius and min_radius > 0:
    radius = int((min_radius + max_radius)/2)
    eps = radius/m_per_radian
    #clustering = DBSCAN(eps=eps, algorithm='ball_tree', metric='precomputed')
    clustering = DBSCAN(eps=eps, metric='precomputed')
    clustering.fit(mat1)
    #n_clusters, n_noise = num_clusters(clustering.labels_)
    max_cluster_size, avg_cluster_size = cluster_size(clustering.labels_)
    #print(eps,max_cluster_size)
    if max_cluster_size > max_cluster_size_const:
        max_radius = radius - 1
    else:
        min_radius = radius + 1
        if avg_cluster_size > best_avg_cluster_size:
            best_avg_cluster_size = avg_cluster_size
            final_clusters = clustering.labels_
if final_clusters is not None:
    print(final_clusters)
else:
    print("No solution found")

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  3 -1  3  3
  3  3  3  3  3  3  3  3  0  0  2  2  2  2  0  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2]


Outliers ignored for now in the _generateClusters_ function

In [43]:
def generate_clusters(labels):
    clusters = {}
    for i in range(len(labels)):
        if labels[i] == -1:
            continue
        if labels[i] in clusters:
            clusters[labels[i]].append(i)
        else:
            clusters[labels[i]] = [i]
    return clusters

clusters = generate_clusters(final_clusters)

In [44]:
def construct_distance_matrix(points):
    n = len(points)
    cluster_distance_matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            cluster_distance_matrix[i,j] = mat1[points[i], points[j]]
    return cluster_distance_matrix

In [45]:
def ortools_optimise_clusters(clusters, vehicles, toUseVehicles):
    for cluster_num, cluster_points in clusters.items():
        cluster_distance_matrix = construct_distance_matrix(cluster_points)
        if vehicles <= 0:
            print("No solution found")
            return
        optimizer = ORTools_Optimizer(cluster_distance_matrix, toUseVehicles[cluster_num], cluster_points)
        optimizer.run()
        vehicles -= toUseVehicles[cluster_num]

In [47]:
ortools_optimise_clusters(clusters, 5, [1,1,2,1])

Objective: 332070033207
Route for vehicle 0:
 0 ->  158 ->  28 ->  29 ->  18 ->  5 ->  27 ->  22 ->  39 ->  26 ->  47 ->  152 ->  153 ->  12 ->  13 ->  19 ->  30 ->  45 ->  17 ->  24 ->  41 ->  23 ->  8 ->  43 ->  38 ->  36 ->  35 ->  34 ->  32 ->  15 ->  16 ->  33 ->  42 ->  4 ->  44 ->  3 ->  10 ->  21 ->  7 ->  6 ->  2 ->  11 ->  9 ->  25 ->  20 ->  46 ->  40 ->  1 ->  37 ->  31 -> 0
Distance of the route: 33207m

Maximum of the route distances: 33207m
Objective: 271550027155
Route for vehicle 0:
 14 ->  48 ->  96 ->  100 ->  95 ->  126 ->  137 ->  97 ->  103 ->  98 ->  113 ->  102 ->  107 ->  104 ->  101 ->  106 ->  129 ->  99 ->  105 ->  123 ->  108 ->  110 ->  118 ->  122 ->  114 ->  130 ->  134 ->  111 ->  136 ->  124 ->  127 ->  135 ->  116 ->  131 ->  115 ->  138 ->  128 ->  132 ->  133 ->  119 ->  125 ->  112 ->  117 ->  109 ->  139 ->  121 ->  120 -> 0
Distance of the route: 27155m

Maximum of the route distances: 27155m
Objective: 363830072172
Route for vehicle 0:
 49 ->  8

# TESTING GROUND

In [ ]:
from sklearn.cluster import DBSCAN

clustering = DBSCAN(eps = 4000, metric='precomputed')
clustering.fit(mat1)

DBSCAN(eps=4000, metric='precomputed')

In [ ]:
clustering.labels_

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  3, -1,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  0,
        0,  1,  1,  1,  1,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  1])

In [ ]:
distances = {}
for i in distance_matrix:
    for j in i:
        if j in distances:
            distances[j] += 1
        else:
            distances[j] = 1
        mini = min(mini, j)
        maxi = max(maxi, j)

In [ ]:
print(mini)
print(maxi)
print(distances)

0
30249
{0: 285, 1799: 2, 3527: 2, 1988: 8, 2142: 2, 3609: 2, 3236: 2, 3643: 2, 2278: 4, 2766: 2, 2624: 4, 3098: 2, 3535: 2, 3372: 2, 7505: 4, 2732: 2, 2803: 6, 3306: 2, 3768: 4, 2671: 2, 3479: 2, 3550: 4, 2173: 2, 3203: 4, 2802: 2, 3689: 2, 3218: 2, 3267: 2, 3067: 2, 1715: 12, 2358: 12, 2243: 6, 1749: 4, 3281: 2, 2154: 6, 2396: 6, 2040: 2, 3012: 2, 2886: 4, 8128: 2, 6321: 2, 9309: 10, 9104: 2, 9832: 8, 9363: 6, 11996: 6, 10058: 4, 9632: 6, 9634: 6, 11771: 4, 10329: 14, 9186: 8, 9605: 8, 8946: 8, 9669: 2, 10003: 4, 8640: 16, 9439: 2, 9982: 2, 10022: 8, 9884: 2, 9905: 8, 8771: 2, 9811: 12, 9719: 8, 9999: 4, 12938: 4, 9626: 4, 9429: 4, 9740: 14, 9224: 2, 9704: 8, 10563: 4, 9709: 6, 9852: 18, 9668: 2, 9961: 12, 9897: 8, 9649: 2, 9983: 6, 8937: 4, 10078: 8, 8842: 8, 9953: 4, 9943: 4, 10016: 8, 10037: 4, 10596: 2, 9973: 8, 10200: 2, 10090: 4, 9534: 6, 10238: 4, 9888: 6, 10479: 6, 10185: 6, 10397: 8, 11471: 8, 12794: 2, 13205: 2, 11953: 4, 13244: 4, 12952: 6, 13170: 2, 12244: 4, 13225: 2, 12

In [20]:
def create_data_model(mat):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = mat
    data['num_vehicles'] = 5
    data['depot'] = 0
    return data

In [21]:
data = create_data_model(distance_matrix)

In [22]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

In [23]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

In [24]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [26]:
# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    30000000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(10000000)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
else:
    print('No solution found !')

Objective: 556860272104
Route for vehicle 0:
 0 ->  102 ->  113 ->  98 ->  103 ->  97 ->  137 ->  126 ->  95 ->  100 ->  96 ->  99 ->  129 ->  105 ->  123 ->  106 ->  101 ->  104 ->  107 ->  108 ->  110 ->  122 ->  118 ->  130 ->  135 ->  116 ->  131 ->  132 ->  133 ->  119 ->  125 ->  120 ->  139 ->  121 ->  109 ->  79 ->  54 ->  60 ->  62 ->  50 ->  61 ->  63 ->  70 ->  82 ->  88 ->  86 ->  67 ->  81 ->  83 ->  73 ->  93 ->  89 ->  85 ->  76 ->  59 ->  64 ->  68 ->  69 ->  51 ->  84 ->  72 ->  56 ->  55 ->  77 ->  66 ->  47 ->  12 ->  10 ->  4 ->  23 ->  41 ->  33 ->  38 ->  36 ->  35 ->  34 ->  32 ->  43 ->  8 ->  26 ->  39 ->  22 ->  27 ->  5 ->  18 ->  29 ->  28 ->  158 -> 0
Distance of the route: 53877m

Route for vehicle 1:
 0 ->  25 ->  87 ->  49 ->  58 ->  52 ->  74 ->  90 ->  92 ->  53 ->  182 ->  184 ->  190 ->  154 ->  178 ->  180 ->  167 ->  172 ->  173 ->  161 ->  166 ->  177 ->  163 ->  174 ->  185 ->  175 ->  192 ->  170 ->  183 ->  57 ->  75 ->  94 ->  78 ->  80 ->  91